<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/reranking_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq sentence-transformers cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [3]:
import sqlite3
from sentence_transformers import SentenceTransformer, util
import numpy as np
import cohere

In [4]:
co = cohere.Client("5xvQOPxJqxpUX03AQRONJ1MJTG9knBBTCMHyFjFg")

In [5]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
DB = '/content/drive/MyDrive/12-4-24-all-mini-token-100.db'

In [7]:
def find_top_k_relevance(question=["No question"], model=None, n=20, db=DB):
  q = model.encode(question)
  scores = {}
  conn = sqlite3.connect(db)

# Create a cursor object
  cursor = conn.cursor()
  # load row by row
  cursor.execute('SELECT * FROM documents')
  for row in cursor:
    scores[row[0]] = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()

  return dict(sorted(scores.items(), key = lambda x: x[1], reverse = True)[:n])


top_k = find_top_k_relevance(question =["How should I store Bananas?"], model=model)

In [8]:
y = list(top_k.keys())

In [47]:
conn = sqlite3.connect(DB)

docs = []
# Create a cursor object
cursor = conn.cursor()
# load row by row
# have to do the comprehension to unpakc inside and f-string
# this doesn't get then in order
for i in y:
    docs.append(cursor.execute(f"""SELECT sentence FROM documents
                  WHERE id = {i}""").fetchone())

docs = [i[0] for i in docs]

In [48]:
q = "How should I store Bananas?"
results = co.rerank(model = "rerank-english-v3.0", query=q, documents=docs, top_n=4, return_documents=True)

In [57]:
relevant_ids = []

for idx, r in enumerate(results.results):
    # only get the best ones..
    if r.relevance_score > 0.7:
        print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
        relevant_ids.append(r.index)
        print(f"Document: {r.document.text}")
        print(f"Relevance Score: {r.relevance_score:.2f}")
        print("\n")

Document Rank: 1, Document Index: 0
Document: . Visual quality was maintained longer where the fruit was kept in polyethylene bags. It is not appropriate to store bananas in the fridge as they are chilling-sensitive, and their skins become blackened, but the coolest place in the home is the best storage location and the fruit should be kept in bags. Bananas sold in the UK are mostly sourced from Central and South America. They are harvested while still in a pre-climacteric (green) condition and shipped to the UK where
Relevance Score: 0.99


Document Rank: 2, Document Index: 10
Document:  bags, and could, after further research, act as an alternative to conventional plastic (Ahmed, 2016).  Speed of ripening aside, a further benefit of packaging is to protect bananas from bruising and excessive handling by citizens. Retailers have reported that damage is caused by citizens breaking bunches of loose bananas while selecting the appropriate number of fruits and their preferred level of rip

In [60]:
db_ids = [y[i] for i in relevant_ids]

### Need to get the para numbers

In [63]:
rel_documents = []
prompt_candidates = []

conn = sqlite3.connect(DB)
cursor = conn.cursor()
for i in db_ids:
    x = cursor.execute(f"""SELECT * FROM documents
                    WHERE id = {i}""").fetchone()
    rel_documents.append(x[2])
    prompt_candidates.append(x[3])

conn.close()

In [67]:
# we would only need to know the unique documents that the text come from..
set(rel_documents) ,prompt_candidates

({'14 - WRAP-Evidence Review Plastic Packaging and Fresh Produce 171218.pdf'},
 ['. Visual quality was maintained longer where the fruit was kept in polyethylene bags. It is not appropriate to store bananas in the fridge as they are chilling-sensitive, and their skins become blackened, but the coolest place in the home is the best storage location and the fruit should be kept in bags. Bananas sold in the UK are mostly sourced from Central and South America. They are harvested while still in a pre-climacteric (green) condition and shipped to the UK where',
  ' bags, and could, after further research, act as an alternative to conventional plastic (Ahmed, 2016).  Speed of ripening aside, a further benefit of packaging is to protect bananas from bruising and excessive handling by citizens. Retailers have reported that damage is caused by citizens breaking bunches of loose bananas while selecting the appropriate number of fruits and their preferred level of ripeness (WRAP, 2011). The damage